In [2]:
!pip install python-binance 

     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 288 kB 10.3 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 103 kB 46.4 MB/s 
     |████████████████████████████████| 1.1 MB 67.5 MB/s 
     |████████████████████████████████| 160 kB 45.5 MB/s 
     |████████████████████████████████| 271 kB 49.9 MB/s 
     |████████████████████████████████| 192 kB 42.6 MB/s 


In [3]:
import pandas as pd
from binance.client import Client
import asyncio
from binance import AsyncClient, BinanceSocketManager
import nest_asyncio
nest_asyncio.apply()

from sqlalchemy import create_engine

In [4]:
engine = create_engine('sqlite:///Crypto.db')

In [6]:
client=Client()
info=client.get_exchange_info()
symbols = [x['symbol']for x in info['symbols']]

In [9]:
exclude = ['UP','DOWN','BEAR','BULL']
non_lev = [symbol for symbol in symbols if all(excludes not in symbol for excludes in exclude)]
relevant = [symbol for symbol in non_lev if symbol.endswith('USDT')]

In [10]:
multi = [i.lower() + '@trade' for i in relevant]

In [12]:
def createframe(msg):
  df = pd.DataFrame([msg['data']])
  df = df.loc[:,['s','E','p']]
  df.columns = ['symbol','Time','Price']
  df.Price = df.Price.astype(float)
  df.Time = pd.to_datetime(df.Time,unit='ms')
  return df

In [19]:
async def main():
  client = await AsyncClient.create()
  bm = BinanceSocketManager(client)
  ms = bm.multiplex_socket(multi)
  async with ms as tscm:
    while True:
      res= await tscm.recv()
      if res :
        frame = createframe(res)
        frame.to_sql(frame.symbol[0],engine, if_exists='append',index=False)

  await client;close_connexion()


In [1]:
if __name__ == "__main__":
  loop = asyncio.get_event_loop()
  loop.run_until_complete(main())

NameError: ignored

In [ ]:
api_key = ???
api_secret = ???


In [29]:
import datetime as dt

In [30]:
client = Client(api_key,api_secret)

In [33]:
symbols = pd.read_sql('SELECT name FROM sqlite_master WHERE type ="table"', engine).name.to_list()

In [48]:
def qry(symbol, lookback):
  now = dt.datetime.now()
  before = now - dt.timedelta(minutes=lookback)
  qry_str = f"""SELECT * FROM '{symbol}' WHERE TIME >='{before}'"""
  return pd.read_sql(qry_str,engine)

In [ ]:
dt.datetime.now()-dt.timedelta(hours=1)

In [ ]:
qry('BTCUSDT',5)


In [56]:
rets=[]
for symbol in symbols:
  prices = qry(symbol,3).Price
  cumret = (prices.pct_change() +1).prod() -1
  rets.append(cumret)

In [58]:
top_coin = symbols[rets.index(max(rets))]

In [62]:
top_coin

'ADAUSDT'

In [70]:
investment_amt = 300
info=client.get_symbol_info(symbol=top_coin)
Lotsize = [i for i in info['filters'] if i['filterType'] == 'LOT_SIZE'][0]['minQty']

In [66]:
prize = float(client.get_symbol_ticker(symbol=top_coin)['price'])

In [68]:
prize

1.262

In [71]:
Lotsize

'0.10000000'

In [72]:
buy_quantity = round(investment_amt/prize, len(str(Lotsize).split('.')[1]))

In [73]:
buy_quantity

237.71790808

In [ ]:
free_usd = [i for i in client.get_account()['balances'] if i['asset'] == 'USDT'][0]['free']

In [ ]:
if float(free_usd)> investment_amt:
  order = client.create_order(symbol=top_coin,side='BUY',
                              type='MARKET',quantity=buy_quantity)
  print(order)
else:
  print('L ordre n a pas été executé')
  quit()

In [ ]:
buyprice = float(order['fills'][0]['price'])


In [ ]:
async def main():
  client = await AsyncClient.create()
  bm = BinanceSocketManager(client)
  ms = bm.multiplex_socket(multi)
  async with ms as tscm:
    while True:
      res= await tscm.recv()
      if res :
        frame = createframe(res)
        if frame.Price[0] < buyprice * 0.97 or frame.Price[0] > 1.005 * buyprice:
          order = client;create_order(symbol=coin,
                                      side='SELL',
                                      type="MARKET",
                                      quantity=buy_quantity)
          print(order)
          loop.stop()

  await client;close_connexion()



In [ ]:
if __name__ == "__main__":
  loop = asyncio.get_event_loop()
  loop.run_until_complete(main(top_coin))